In [ ]:
# Suppress warnings and Pylance diagnostics for this notebook (focus on other subprojects)
# pyright: reportAttributeAccessIssue=false, reportIndexIssue=false, reportOperatorIssue=false
# pyright: reportGeneralTypeIssues=false, reportCallIssue=false, reportArgumentType=false
# pyright: reportOptionalOperand=false, reportOptionalSubscript=false, reportUnusedImport=false

import warnings
warnings.filterwarnings('ignore')

# Week3

## DCOPF 建模

### PTDF 分布模型验证
根据《数据驱动最优潮流的误差可信量化及其抑制方法》式 2.2 至 2.5，DCOPF 具体建模如下

\begin{aligned}
\min_{P_G}\quad & C_G^{\!\top} P_G \qquad\text{(linear objective 线性目标)} \\ \text{s.t.}\quad & \mathbf{1}_G^{\!\top} P_G = \mathbf{1}_D^{\!\top} P_D \quad\text{(system balance 系统功率平衡)} \\ & P_L^{\min} \ \le\ M_G P_G + M_D P_D \ \le\ P_L^{\max} \quad\text{(line limits 线路约束)} \\ & P_G^{\min} \ \le\ P_G \ \le\ P_G^{\max} \quad\text{(generator bounds 机组约束).}
\end{aligned}
 

In [2]:
# pyright: reportAttributeAccessIssue=false, reportIndexIssue=false, reportOperatorIssue=false, reportGeneralTypeIssues=false, reportCallIssue=false, reportArgumentType=false, reportOptionalOperand=false, reportOptionalSubscript=false
import pyomo.environ as pyo
from typing import Any

def build_dc_opf_model() -> Any:
    """Abstract DC-OPF model (variables: P_G)
    Obj: minimize generation cost
    Constraints: system balance, line limits and generator bounds.
    """
    m: Any = pyo.AbstractModel()

    # --------- Sets ---------
    m.G = pyo.Set(doc="Generators")
    m.D = pyo.Set(doc="Demands")
    m.L = pyo.Set(doc="Lines") 

    # --------- Parameters ---------
    # Linear generation cost coefficients
    m.c2 = pyo.Param(m.G, within=pyo.Reals, doc="Linear cost coefficient C_G")
    m.c1 = pyo.Param(m.G, within=pyo.Reals, doc="Linear cost coefficient")
    m.c0 = pyo.Param(m.G, within=pyo.Reals, doc="Constant cost coefficient")

    # Demand vector (indexed by m.D)
    m.Pd = pyo.Param(m.D, doc="Demand at each demand node (fixed)")

    # Line flow limits
    m.P_L_max = pyo.Param(m.L, within=pyo.Reals, doc="Line flow upper limit")
    m.P_L_min = pyo.Param(m.L, within=pyo.Reals, doc="Line flow lower limit")

    # PTDFs: contribution of each generator / demand to line flows
    m.M_G = pyo.Param(m.L, m.G, within=pyo.Reals, doc="PTDFs for generators")
    m.M_D = pyo.Param(m.L, m.D, within=pyo.Reals, doc="PTDFs for demands")

    # Generator output bounds
    m.P_G_max = pyo.Param(m.G, within=pyo.Reals, doc="Generator max output")
    m.P_G_min = pyo.Param(m.G, within=pyo.Reals, doc="Generator min output")

    # --------- Variables ---------
    def _P_G_bounds(m: Any, g):
        return (m.P_G_min[g], m.P_G_max[g])
    m.P_G = pyo.Var(m.G, bounds=_P_G_bounds, doc="Generator outputs (MW)")

    # --------- Objective ---------
    def _gen_cost_rule(m: Any):
        return sum(m.c2[g] * m.P_G[g]**2 + m.c1[g] * m.P_G[g] + m.c0[g] for g in m.G)
    m.Obj = pyo.Objective(rule=_gen_cost_rule, sense=pyo.minimize)

    # --------- Constraints ---------
    # System balance: sum_g P_G = sum_d P_d
    def _system_balance_rule(m: Any):
        return sum(m.P_G[g] for g in m.G) == sum(m.Pd[d] for d in m.D)
    m.System_Balance = pyo.Constraint(rule=_system_balance_rule)

     # Line flow expression: f_l = M_G*P_G + M_D*P_d
    def _flow_def(m: Any, l):
        return sum(m.M_G[l, g]*m.P_G[g] for g in m.G) \
             + sum(m.M_D[l, d]*m.Pd[d] for d in m.D)
    m.Flow = pyo.Expression(m.L, rule=_flow_def)

    # Line limits: P_L_min <= f_l <= P_L_max
    def _flow_limits(m: Any, l):
        return pyo.inequality(m.P_L_min[l], m.Flow[l], m.P_L_max[l])
    m.LineLimits = pyo.Constraint(m.L, rule=_flow_limits)

    return m


In [3]:
# pyright: reportAttributeAccessIssue=false, reportIndexIssue=false, reportOperatorIssue=false, reportGeneralTypeIssues=false, reportCallIssue=false, reportArgumentType=false, reportOptionalOperand=false, reportOptionalSubscript=false
import numpy as np
import pyomo.environ as pyo

# ---- helpers to build PTDF & Pyomo data from a PYPOWER case ----

def _bus_index_maps(ppc):
    bus = ppc["bus"]
    # PYPOWER uses 1-based bus numbers in the data; build map -> 0-based row indices
    bus_nums = bus[:, 0].astype(int)
    idx_of_busnum = {int(bn): i for i, bn in enumerate(bus_nums)}
    return bus_nums, idx_of_busnum

def _choose_slack(ppc, idx_of_busnum):
    # type column is 2nd field (index 1): 3=REF, 2=PV, 1=PQ
    REF = 3
    bus = ppc["bus"]
    ref_rows = np.where(bus[:,1] == REF)[0]
    if len(ref_rows) == 0:
        # fallback: smallest bus number
        return 0
    return int(ref_rows[0])

def _build_dc_matrices(ppc, slack_row):
    """Return (Bbus, Cf, b_line) with tap ratios accounted for, angles ignored."""
    bus = ppc["bus"]
    branch = ppc["branch"]

    nb = bus.shape[0]
    nl = branch.shape[0]

    # columns in branch: fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
    FBUS, TBUS, R, X, B, RATEA, RATEB, RATEC, RATIO, ANGLE, STATUS, ANGMIN, ANGMAX = range(13)

    # active lines
    on = branch[:, STATUS] > 0.5
    br = branch[on, :]

    # susceptance per branch (p.u. on system base); DC uses 1/x, include tap ratio
    x = br[:, X]
    if np.any(np.isclose(x, 0.0)):
        raise ValueError("Branch reactance x=0 found; DC PTDF undefined for that branch.")
    tap = br[:, RATIO].copy()
    tap[ (tap==0) | np.isnan(tap) ] = 1.0  # default tap=1
    b_series = 1.0 / x
    b_line = b_series / tap  # effective susceptance on "from" side

    # incidence matrix Cf (nl_on x nb): +1 at from, -1 at to
    fbus = br[:, FBUS].astype(int)
    tbus = br[:, TBUS].astype(int)

    # map bus numbers -> row indices
    bus_nums, idx_map = _bus_index_maps(ppc)
    nl_on = br.shape[0]
    Cf = np.zeros((nl_on, nb))
    for k in range(nl_on):
        i = idx_map[int(fbus[k])]
        j = idx_map[int(tbus[k])]
        Cf[k, i] =  1.0 / tap[k]   # tap on from-side
        Cf[k, j] = -1.0

    # Bf and Bbus
    Bf   = (b_line[:, None] * Cf)                   # f = Bf * theta
    Bbus = Cf.T @ (b_line[:, None] * Cf)            # p = Bbus * theta

    # Reduce by removing slack row/col for inversion
    keep = [i for i in range(nb) if i != slack_row]
    Bbus_red = Bbus[np.ix_(keep, keep)]

    return Bbus, Bbus_red, Bf, Cf, b_line, keep

def _compute_ptdf(ppc):
    """Compute PTDF (lines × buses) w.r.t. chosen slack."""
    _, idx_map = _bus_index_maps(ppc)
    slack_row = _choose_slack(ppc, idx_map)
    Bbus, Bbus_red, Bf, Cf, _, keep = _build_dc_matrices(ppc, slack_row)

    # PTDF via columns: for each bus k, inject +1 at k and -1 at slack
    nb = Bbus.shape[0]
    nl = Bf.shape[0]
    H = np.zeros((nl, nb))  # PTDF[:, bus]

    # Pre-factorize reduced Bbus for speed
    # (Cholesky may fail if singular; use np.linalg.solve)
    for k in range(nb):
        if k == slack_row:
            # by definition, a 1 p.u. injection at slack balanced at slack gives zero net — column zeros
            H[:, k] = 0.0
            continue
        # build p_red: +1 at k, -1 at slack -> in reduced system it's +1 at k, slack removed
        p = np.zeros(nb)
        p[k] =  1.0
        p[slack_row] = -1.0
        p_red = p[keep]
        theta_red = np.linalg.solve(Bbus_red, p_red)

        # reconstruct full theta (insert 0 at slack position)
        theta = np.zeros(nb)
        theta[keep] = theta_red
        theta[slack_row] = 0.0

        # line flows for that unit transaction
        f = Bf @ theta
        H[:, k] = f

    # Only rows for in-service branches were built; map them back to full list of lines if needed
    return H, slack_row

def ppc_to_pyomo_data_for_create_instance(ppc):
    """Return (data_dict, meta) where data_dict matches Pyomo create_instance(data=...)."""
    baseMVA = ppc["baseMVA"]
    bus = ppc["bus"]; gen = ppc["gen"]; branch = ppc["branch"]; gencost = ppc["gencost"]

    bus_nums, idx_map = _bus_index_maps(ppc)
    nb = bus.shape[0]; ng = gen.shape[0]; nl = branch.shape[0]

    # --- Sets (string labels) ---
    G = [f"g{g}" for g in range(ng)]
    PD_COL = 2  # Pd col in bus
    load_rows = [i for i in range(nb) if bus[i, PD_COL] > 1e-9]
    D = [f"d{bus_nums[i]}" for i in load_rows]
    L = [f"l{k}" for k in range(nl)]

    # --- Gen bounds & mapping gen->bus row ---
    PMAX, PMIN, GBUS = 8, 9, 0
    Pg_max = {G[g]: float(gen[g, PMAX]) for g in range(ng)}
    Pg_min = {G[g]: float(gen[g, PMIN]) for g in range(ng)}
    g_bus_rows = [idx_map[int(gen[g, GBUS])] for g in range(ng)]

    # --- Quadratic costs (type 2) ---
    c2 = {}; c1 = {}; c0 = {}
    for g in range(ng):
        c2[G[g]] = float(gencost[g, 4])
        c1[G[g]] = float(gencost[g, 5])
        c0[G[g]] = float(gencost[g, 6])

    # --- Demands ---
    Pd = {f"d{bus_nums[i]}": float(bus[i, PD_COL]) for i in load_rows}

    # --- Line limits ---
    RATEA, STATUS = 5, 10
    bigM = 1e6
    P_L_max = {}
    P_L_min = {}
    active_mask = branch[:, STATUS] > 0.5
    for k in range(nl):
        lim = float(branch[k, RATEA]) if (active_mask[k] and branch[k, RATEA] > 0) else bigM
        P_L_max[f"l{k}"] = lim
        P_L_min[f"l{k}"] = -lim

    # --- PTDF (lines_in_service × buses) ---
    H, slack_row = _compute_ptdf(ppc)

    # Map PTDF back to all lines (zeros for out-of-service)
    on = np.where(active_mask)[0]
    H_full = np.zeros((nl, nb))
    H_full[on, :] = H

    # --- Build M_G and M_D ---
    M_G = {}
    for g in range(ng):
        col = H_full[:, g_bus_rows[g]]
        for k in range(nl):
            M_G[(f"l{k}", G[g])] = float(col[k])

    M_D = {}
    for i in load_rows:
        dkey = f"d{bus_nums[i]}"
        col = H_full[:, i]
        for k in range(nl):
            M_D[(f"l{k}", dkey)] = float(-col[k])  # minus for loads

    # --- Pack in Pyomo's required shapes ---
    data_dict = {
        # Sets must be wrapped as {None: iterable}
        "G": {None: list(G)},
        "D": {None: list(D)},
        "L": {None: list(L)},

        # 1-D Params (keys match model component names)
        "P_G_min": Pg_min,
        "P_G_max": Pg_max,
        "c2": c2, "c1": c1, "c0": c0,
        "Pd": Pd,
        "P_L_min": P_L_min,
        "P_L_max": P_L_max,

        # 2-D Params: keys are tuples of indices used by the model
        "M_G": M_G,
        "M_D": M_D,
    }

    inner = {
        "G": {None: list(G)},
        "D": {None: list(D)},
        "L": {None: list(L)},
        "P_G_min": Pg_min,
        "P_G_max": Pg_max,
        "c2": c2, "c1": c1, "c0": c0,
        "Pd": Pd,
        "P_L_min": P_L_min, "P_L_max": P_L_max,
        "M_G": M_G,
        "M_D": M_D,
    }
    data_wrapped = {None: inner}
    meta = {"slack_row": int(slack_row), "baseMVA": float(baseMVA)}
    return data_wrapped, meta


In [4]:
# pyright: reportOptionalOperand=false, reportCallIssue=false, reportArgumentType=false, reportOptionalSubscript=false
from typing import Optional

# Load case and solve DCOPF
from pypower.api import case118

ppc = case118()
model = build_dc_opf_model()

data_dict, meta = ppc_to_pyomo_data_for_create_instance(ppc)
instance = model.create_instance(data=data_dict)   # now OK

opt = pyo.SolverFactory('ipopt')
res = opt.solve(instance, tee=False)


print('Solver termination:', res.solver.termination_condition)
print('Objective value: ${:,.2f}/hr'.format(pyo.value(instance.Obj)))

# Display generator dispatch results
print(f'\nGenerator dispatch (showing active generators):')
print(f'{"Generator":<12} {"Output (MW)":>12}')
print('-' * 26)

active_count = 0
for g in sorted(list(instance.G), key=lambda x: int(str(x)[1:])):
    Pg_val_raw: Optional[float] = pyo.value(instance.P_G[g])
    if Pg_val_raw is None:
        continue
    Pg_val: float = float(Pg_val_raw)
    # Only show generators with non-trivial output
    if Pg_val > 0.01:
        print(f'{g:<12} {Pg_val:12.2f}')
        active_count += 1

print('-' * 26)
nG = len(list(instance.G))
total_generation: float = sum(float(pyo.value(instance.P_G[g]) or 0.0) for g in instance.G)
print(f'Active generators: {active_count} of {nG}')
print(f'Total generation: {total_generation:,.2f} MW')


Solver termination: optimal
Objective value: $125,947.87/hr

Generator dispatch (showing active generators):
Generator     Output (MW)
--------------------------
g4                 436.08
g5                  82.37
g10                213.20
g11                304.29
g13                  6.78
g19                 18.41
g20                197.69
g21                 46.52
g24                150.21
g25                155.05
g27                378.91
g28                379.87
g29                500.43
g36                462.24
g38                  3.88
g39                588.22
g44                244.21
g45                 38.76
g50                 34.89
--------------------------
Active generators: 19 of 54
Total generation: 4,242.00 MW


In [5]:
# pyright: reportOptionalSubscript=false, reportArgumentType=false
from typing import Any, Iterable

# --- Run DCOPF using PYPOWER built-in solver -----------------
from pypower.api import case118, rundcopf

# Load the case
ppc = case118()

# Run DC-OPF
results = rundcopf(ppc)

# ---- Print generator outputs ----
print("\nGenerator dispatch results (MW):")
# Safely access generator matrix from results without ambiguous truth checks
gens_val = results.get("gen") if isinstance(results, dict) else None
if gens_val is None:
    gens_iter: Iterable[Any] = []
else:
    gens_iter = gens_val

# Column 0 = bus number, 1 = Pg, 8 = Pmax in MATPOWER/PYPOWER format
for i, row in enumerate(gens_iter, start=1):
    bus_num = int(row[0])
    Pg = row[1]
    Pmax = row[8]
    print(f"  Gen {i:02d} at Bus {bus_num:3d}:  Pg = {Pg:10.4f} MW   (Pmax = {Pmax:10.4f})")

PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.04 seconds
Objective Function Value = 125947.87 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses            118     Total Gen Capacity    9966.2           0.0 to 0.0
Generators        54     On-line Capacity      9966.2           0.0 to 0.0
Committed Gens    54     Generation (actual)   4242.0               0.0
Loads             99     Load                  4242.0               0.0
  Fixed           99       Fixed               4242.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches         186     Losses (I^2 * Z)         0

### 从 `.xlsx` 文件中提取 $P_G^{min}$ and $P_G^{max}$ 

In [6]:
import pandas as pd
from pathlib import Path

def load_generator_limits(xlsx_path, sheet_index=2):
    """
    Read generator limits from the third sheet of an Excel file.

    Parameters
    ----------
    xlsx_path : str
        Path to your .xlsx file
    sheet_index : int
        Sheet index (0-based). The 3rd sheet → 2.

    Returns
    -------
    gen_bus : ndarray of shape (ng,)
        Bus numbers of generators (1-based IDs)
    PGmax : ndarray of shape (ng,)
        Generator upper bounds (p.u.)
    PGmin : ndarray of shape (ng,)
        Generator lower bounds (p.u.)
    """

    df = pd.read_excel(xlsx_path, sheet_name=sheet_index)

    # Normalize column names to be safe
    df.columns = [c.strip() for c in df.columns]

    # Expected column names:
    # 'Gen_Bus', 'Pmax(p.u.)', 'Pmin(p.u.)'
    gen_bus = df['Gen_Bus'].to_numpy(dtype=int)
    PGmax   = df['Pmax(p.u.)'].to_numpy(dtype=float)
    PGmin   = df['Pmin(p.u.)'].to_numpy(dtype=float)

    print(f"Loaded {len(gen_bus)} generator entries from {xlsx_path}")
    return gen_bus, PGmax, PGmin

### 提取数据并进行预处理

In [7]:
# pyright: reportAttributeAccessIssue=false, reportArgumentType=false, reportOperatorIssue=false
import numpy as np
from pypower.api import case118
ppc = case118()

# Extract set G
ng = ppc['gen'].shape[0]  # number of generators
G = np.arange(ng)
"""
gen_bus = ppc['gen'][:, 0].astype(int) - 1  # 0-based bus indices
m.bus_of_gen = pyo.Param(m.G, initialize=lambda m,i: int(gen_bus[i]))
"""

# Extract cost coefficients
gencost = ppc['gencost']
c2 = gencost[:, 4]
c1 = gencost[:, 5]
c0 = gencost[:, 6]

# Extract Set D
nb = ppc["bus"].shape[0]  # number of buses (demands)
D = np.arange(nb)

# Extract Set L
branch = ppc['branch']
nbr = branch.shape[0] # number of branches (lines) sometimes nl in comments
L = np.arange(nbr)

# Extract P_L^{min} and P_L^{max}
PLmax = np.full(nbr, 300.0, dtype=float)     # shape (nbr,)
PLmin = np.full(nbr, -240.0, dtype=float)    # shape (nbr,)

# Extract P_G^{min} and P_G^{max}
xlsx_path = Path.cwd() / "IEEE_118_bus_test_system.xlsx"  # no leading slash
gen_bus, PGmax_pu, PGmin_pu = load_generator_limits(xlsx_path)
# Convert from p.u. to MW: multiply by baseMVA (= 100.0 for case118)
baseMVA = float(ppc["baseMVA"])
PGmax = PGmax_pu * baseMVA  # MW
PGmin = PGmin_pu * baseMVA  # MW
print("Gen bus (first 5):", gen_bus[4:7])
print("PGmax (MW, first 5):", PGmax[4:7])
print("PGmin (MW, first 5):", PGmin[4:7])

# Extract M_G and M_D
FBUS, TBUS, R, X, B, RATEA, RATEB, RATEC, RATIO, ANGLE, STATUS = range(11)
# 1) tap ratios (taps) and series reactances (X)
t = branch[:, RATIO].copy()
t[t == 0] = 1.0                         # defualt tap = 1
x = branch[:, X].copy()
# 2) active-line mask; we’ll build on active rows then expand back
active = (branch[:, STATUS] > 0) & (x != 0.0)
idx_act = np.where(active)[0]
nl_act = idx_act.size
# 3) tap-weighted incidence C_f (active rows only)
Cf = np.zeros((nl_act, nb))
for r, ell in enumerate(idx_act):
    i = int(branch[ell, FBUS]) - 1      # 0-based
    j = int(branch[ell, TBUS]) - 1
    Cf[r, i] =  1.0 / t[ell]
    Cf[r, j] = -1.0
# 4) DC “line couplings”
b = 1.0 / (x[idx_act] * t[idx_act])     # b_ell = 1/(x_ell * t_ell)
Bf = (b[:, None]) * Cf                  # diag(b) @ Cf
# 5) Nodal susceptance and slack reduction
Bbus = Cf.T @ ((b[:, None]) * Cf)       # Cf^T diag(b) Cf  (nb x nb)
slack = 0                               # choose a slack bus (0-based)
keep = [k for k in range(nb) if k != slack]
Bbus_red = Bbus[np.ix_(keep, keep)]
inv_Bbus_red = np.linalg.inv(Bbus_red)
E = np.eye(nb)[:, keep]                 # (nb x (nb-1))
# This constructs the “pinv” that enforces theta_slack = 0
Bbus_pinv = E @ inv_Bbus_red @ E.T      # (nb x nb)
# 6) PTDF for active lines; insert zero rows for out-of-service
H_act = Bf @ Bbus_pinv                  # (nl_act x nb)
H_full = np.zeros((nbr, nb))
H_full[idx_act, :] = H_act              # zeros for inactive lines
# 7) Map to MG, MD
gen_bus_0b = ppc['gen'][:, 0].astype(int) - 1    # length ng
nb_d = nb                                        # we index demands by bus
MG = H_full[:, gen_bus_0b]                       # (nl x ng)
MD = -H_full                                     # (nl x nb) : demand at bus d is -injection

Loaded 54 generator entries from e:\DOCUMENT\Learn_Py\opf\Week3\IEEE_118_bus_test_system.xlsx
Gen bus (first 5): [10 12 15]
PGmax (MW, first 5): [550. 185. 100.]
PGmin (MW, first 5): [0. 0. 0.]


### 嵌入数据的 Concrete Model

In [8]:
# pyright: reportAttributeAccessIssue=false, reportArgumentType=false, reportOperatorIssue=false
import pyomo.environ as pyo
import numpy as np

def build_concrete_dcopf(ng, nb, nl, c2, c1, c0, PGmin, PGmax, PLmin, PLmax, MG, MD, PD_MW):
    """
    Build a Concrete DCOPF Model from the formula:
    
    min_{P_G}  sum(c2*PG^2 + c1*PG + c0)
    s.t.  sum(PG) = sum(PD)                          (system balance)
          PLmin <= M_G*PG + M_D*PD <= PLmax       (line limits)
          PGmin <= PG <= PGmax                     (generator bounds)
    
    Parameters
    ----------
    ng : int
        Number of generators
    nb : int
        Number of buses
    nl : int
        Number of lines
    c2, c1, c0 : array-like, shape (ng,)
        Quadratic, linear, and constant cost coefficients ($/hr, $/MWh, $)
    PGmin, PGmax : array-like, shape (ng,)
        Generator bounds (MW)
    PLmin, PLmax : array-like, shape (nl,)
        Line flow limits (MW)
    MG : array-like, shape (nl, ng)
        PTDF matrix for generators (dimensionless)
    MD : array-like, shape (nl, nb)
        PTDF matrix for demands (dimensionless)
    PD_MW : array-like, shape (nb,)
        Demand at each bus (MW)
    
    Returns
    -------
    m : pyo.ConcreteModel
        The constructed Pyomo model ready to solve
    """
    
    # Build Pyomo ConcreteModel
    m = pyo.ConcreteModel(name="DCOPF_Concrete")
    
    # --- Sets ---
    m.G = pyo.Set(initialize=range(ng), doc="Generator indices")
    m.D = pyo.Set(initialize=range(nb), doc="Bus/demand indices")
    m.L = pyo.Set(initialize=range(nl), doc="Line indices")
    
    # --- Parameters (all in MW or dimensionless) ---
    m.c2 = pyo.Param(m.G, initialize={g: float(c2[g]) for g in range(ng)}, doc="Quadratic cost coeff")
    m.c1 = pyo.Param(m.G, initialize={g: float(c1[g]) for g in range(ng)}, doc="Linear cost coeff")
    m.c0 = pyo.Param(m.G, initialize={g: float(c0[g]) for g in range(ng)}, doc="Constant cost term")
    
    m.PGmin = pyo.Param(m.G, initialize={g: float(PGmin[g]) for g in range(ng)}, doc="Gen min (MW)")
    m.PGmax = pyo.Param(m.G, initialize={g: float(PGmax[g]) for g in range(ng)}, doc="Gen max (MW)")
    
    m.PLmin = pyo.Param(m.L, initialize={l: float(PLmin[l]) for l in range(nl)}, doc="Line min (MW)")
    m.PLmax = pyo.Param(m.L, initialize={l: float(PLmax[l]) for l in range(nl)}, doc="Line max (MW)")
    
    # PTDFs (dimensionless)
    m.MG = pyo.Param(m.L, m.G, initialize={
        (l, g): float(MG[l, g]) for l in range(nl) for g in range(ng)
    }, doc="PTDF for generators")
    
    m.MD = pyo.Param(m.L, m.D, initialize={
        (l, d): float(MD[l, d]) for l in range(nl) for d in range(nb)
    }, doc="PTDF for demands")
    
    # Demand (MW, mutable for future scenarios)
    m.PD = pyo.Param(m.D, initialize={d: float(PD_MW[d]) for d in range(nb)}, 
                     mutable=True, doc="Demand at each bus (MW)")
    
    # --- Variables ---
    def pg_bounds_rule(m, g):
        return (m.PGmin[g], m.PGmax[g])
    
    m.PG = pyo.Var(m.G, bounds=pg_bounds_rule, doc="Generator output (MW)")
    
    # --- Objective: Quadratic generation cost ---
    def obj_rule(m):
        return pyo.quicksum(
            m.c2[g] * m.PG[g]**2 + m.c1[g] * m.PG[g] + m.c0[g] 
            for g in m.G
        )
    
    m.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize, doc="Total generation cost ($/hr)")
    
    # --- Constraint 1: System balance (sum PG = sum PD) ---
    def balance_rule(m):
        return pyo.quicksum(m.PG[g] for g in m.G) == pyo.quicksum(m.PD[d] for d in m.D)
    
    m.Balance = pyo.Constraint(rule=balance_rule, doc="Power balance")
    
    # --- Constraint 2: Line flow limits (PLmin <= M_G*PG + M_D*PD <= PLmax) ---
    def line_flow_rule(m, l):
        flow = pyo.quicksum(m.MG[l, g] * m.PG[g] for g in m.G) + \
               pyo.quicksum(m.MD[l, d] * m.PD[d] for d in m.D)
        return pyo.inequality(m.PLmin[l], flow, m.PLmax[l])
    
    m.LineFlow = pyo.Constraint(m.L, rule=line_flow_rule, doc="Line flow limits")
    
    return m

### 验证模型

In [9]:
# pyright: reportAttributeAccessIssue=false, reportArgumentType=false, reportOperatorIssue=false
# ============================================================================
# Load Case118 Data and Solve Concrete DCOPF with Gurobi
# ============================================================================

# Load demand from case118 (MW)
baseMVA = float(ppc['baseMVA'])
PD_MW = ppc['bus'][:, 2]  # column 2 is Pd in MW

print(f"Building Concrete DCOPF Model:")
print(f"  Generators: {ng}")
print(f"  Buses (demands): {nb}")
print(f"  Lines: {nbr}")
print(f"  Total demand: {PD_MW.sum():.2f} MW")

# Build the model using data from cell 9
m = build_concrete_dcopf(ng, nb, nbr, c2, c1, c0, PGmin, PGmax, PLmin, PLmax, MG, MD, PD_MW)

print("\nModel construction complete.")
print(f"  Variables: {len(list(m.G))} PG")
print(f"  Constraints: 1 balance + {len(list(m.L))} line limits = {1 + len(list(m.L))}")

# ============================================================================
# Solve with Gurobi
# ============================================================================
print("\n" + "="*70)
print("SOLVING WITH GUROBI")
print("="*70)

solver = pyo.SolverFactory('gurobi')
results = solver.solve(m, tee=True)

# ============================================================================
# Display Results
# ============================================================================
print("\n" + "="*70)
print("SOLUTION SUMMARY")
print("="*70)
print(f"Solver Status: {results.solver.status}")
print(f"Termination Condition: {results.solver.termination_condition}")

if results.solver.termination_condition == pyo.TerminationCondition.optimal:
    obj_val = pyo.value(m.OBJ)
    print(f"\nObjective Value: ${obj_val:,.2f}/hr")
    
    # Generator dispatch
    print(f"\n{'='*70}")
    print("GENERATOR DISPATCH")
    print(f"{'='*70}")
    print(f"{'Gen':<6} {'Pg (MW)':>12} {'Pmin':>10} {'Pmax':>10} {'c2':>12} {'c1':>12}")
    print("-" * 70)
    
    total_gen = 0.0
    active_count = 0
    
    for g in sorted(m.G):
        pg_val = pyo.value(m.PG[g])
        if pg_val > 1e-3:  # Show generators producing > 1 kW
            print(f"{g:<6} {pg_val:12.2f} {m.PGmin[g]:10.2f} {m.PGmax[g]:10.2f} "
                  f"{m.c2[g]:12.6f} {m.c1[g]:12.2f}")
            total_gen += pg_val
            active_count += 1
    
    print("-" * 70)
    print(f"Active Generators: {active_count}/{ng}")
    print(f"Total Generation: {total_gen:,.2f} MW")
    print(f"Total Demand: {sum(pyo.value(m.PD[d]) for d in m.D):,.2f} MW")
    print(f"Balance Error: {abs(total_gen - sum(pyo.value(m.PD[d]) for d in m.D)):.6f} MW")
    
    # Check line flows
    print(f"\n{'='*70}")
    print("LINE FLOW CHECK (first 10 lines)")
    print(f"{'='*70}")
    print(f"{'Line':<6} {'Flow (MW)':>12} {'Limit (MW)':>12} {'Utilization':>12}")
    print("-" * 70)
    
    for l in list(m.L)[:10]:
        flow = pyo.value(
            pyo.quicksum(m.MG[l, g] * m.PG[g] for g in m.G) +
            pyo.quicksum(m.MD[l, d] * m.PD[d] for d in m.D)
        )
        limit = m.PLmax[l]
        util = abs(flow / limit) * 100 if limit > 0 else 0
        print(f"{l:<6} {flow:12.2f} {limit:12.2f} {util:11.1f}%")
    
    print("="*70)
else:
    print(f"\n⚠ Solver did not find optimal solution!")
    print(f"Termination: {results.solver.termination_condition}")

Building Concrete DCOPF Model:
  Generators: 54
  Buses (demands): 118
  Lines: 186
  Total demand: 4242.00 MW

Model construction complete.
  Variables: 54 PG
  Constraints: 1 balance + 186 line limits = 187

SOLVING WITH GUROBI

Model construction complete.
  Variables: 54 PG
  Constraints: 1 balance + 186 line limits = 187

SOLVING WITH GUROBI
skipping further warning of this kind
Read LP format model from file C:\Users\63091\AppData\Local\Temp\tmpso624cs3.pyomo.lp
Reading time = 0.03 seconds
x1: 373 rows, 54 columns, 17636 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 373 rows, 54 columns and 17636 nonzeros
Model fingerprint: 0x85e921de
Model has 54 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-07, 1e+00]
  Objective range  [2e+0

### 保存参数

In [10]:
# pyright: reportAttributeAccessIssue=false, reportArgumentType=false, reportOperatorIssue=false
# ============================================================================
# Freeze All Parameters - Save to .npz File
# ============================================================================
# Save all model parameters except PD_MW to a .npz file for efficient loading
# during sample generation.

import numpy as np
from pathlib import Path

# Define output path
output_dir = Path.cwd()
frozen_params_file = output_dir / "dcopf_frozen_params.npz"

print("="*70)
print("SAVING FROZEN PARAMETERS")
print("="*70)

# Collect all frozen parameters
frozen_params = {
    # Set dimensions
    'ng': ng,
    'nb': nb,
    'nbr': nbr,
    
    # Set indices (for reference)
    'G': G,
    'D': D,
    'L': L,
    
    # Cost coefficients (ng,)
    'c2': c2,
    'c1': c1,
    'c0': c0,
    
    # Generator limits (ng,)
    'PGmin': PGmin,
    'PGmax': PGmax,
    
    # Line limits (nbr,)
    'PLmin': PLmin,
    'PLmax': PLmax,
    
    # PTDF matrices
    'MG': MG,  # (nbr, ng)
    'MD': MD,  # (nbr, nb)
    
    # Metadata
    'baseMVA': baseMVA,
}

# Save to .npz file
np.savez_compressed(frozen_params_file, **frozen_params)

print(f"\n✓ Frozen parameters saved to:")
print(f"  {frozen_params_file}")
print(f"\nFile contains:")
print(f"  - Set dimensions: ng={ng}, nb={nb}, nbr={nbr}")
print(f"  - Cost coefficients: c2, c1, c0 (shape {c2.shape})")
print(f"  - Generator limits: PGmin, PGmax (shape {PGmin.shape})")
print(f"  - Line limits: PLmin={PLmin[0]:.1f}, PLmax={PLmax[0]:.1f} MW (shape {PLmin.shape})")
print(f"  - PTDF matrices: MG {MG.shape}, MD {MD.shape}")
print(f"  - Base MVA: {baseMVA}")

# Verify file size
file_size_mb = frozen_params_file.stat().st_size / (1024 * 1024)
print(f"\nFile size: {file_size_mb:.2f} MB")

print(f"\n{'='*70}")
print("VARIABLE PARAMETER (Not Saved - Will Vary)")
print(f"{'='*70}")
print(f"\n  PD_MW: Demand vector (shape {PD_MW.shape})")
print(f"    - Current total demand: {PD_MW.sum():.2f} MW")
print(f"    - Will be varied for each sample")

print(f"\n{'='*70}")
print("USAGE: Load frozen parameters with:")
print(f"{'='*70}")
print(f"  data = np.load('dcopf_frozen_params.npz')")
print(f"  ng, nb, nbr = data['ng'], data['nb'], data['nbr']")
print(f"  c2, c1, c0 = data['c2'], data['c1'], data['c0']")
print(f"  PGmin, PGmax = data['PGmin'], data['PGmax']")
print(f"  PLmin, PLmax = data['PLmin'], data['PLmax']")
print(f"  MG, MD = data['MG'], data['MD']")
print(f"  baseMVA = data['baseMVA']")

print(f"\n{'='*70}")
print("STATUS: Ready for sample generation with varying PD_MW")
print(f"{'='*70}")

SAVING FROZEN PARAMETERS

✓ Frozen parameters saved to:
  e:\DOCUMENT\Learn_Py\opf\Week3\dcopf_frozen_params.npz

File contains:
  - Set dimensions: ng=54, nb=118, nbr=186
  - Cost coefficients: c2, c1, c0 (shape (54,))
  - Generator limits: PGmin, PGmax (shape (54,))
  - Line limits: PLmin=-240.0, PLmax=300.0 MW (shape (186,))
  - PTDF matrices: MG (186, 54), MD (186, 118)
  - Base MVA: 100.0

File size: 0.18 MB

VARIABLE PARAMETER (Not Saved - Will Vary)

  PD_MW: Demand vector (shape (118,))
    - Current total demand: 4242.00 MW
    - Will be varied for each sample

USAGE: Load frozen parameters with:
  data = np.load('dcopf_frozen_params.npz')
  ng, nb, nbr = data['ng'], data['nb'], data['nbr']
  c2, c1, c0 = data['c2'], data['c1'], data['c0']
  PGmin, PGmax = data['PGmin'], data['PGmax']
  PLmin, PLmax = data['PLmin'], data['PLmax']
  MG, MD = data['MG'], data['MD']
  baseMVA = data['baseMVA']

STATUS: Ready for sample generation with varying PD_MW


## 样本生成

### 随机生成样本

In [11]:
# pyright: reportAttributeAccessIssue=false, reportArgumentType=false, reportOperatorIssue=false
import numpy as np
import pyomo.environ as pyo
from pathlib import Path
from time import perf_counter
from scipy.stats import qmc

# Dataset generator/labeler with LHS
# - Generates 10000 random load vectors PD_MW using Latin Hypercube Sampling
# - Total demand varies from 60% to 130% of baseline case118
# - Per-bus demands clipped to [0, 150] MW
# - Saved in 40 chunks of 250 samples each

# Tweakables
TOTAL_SAMPLES = 10000
CHUNK_SIZE = 250
PD_LOW = 0.0  # MW (per-bus lower bound, enforced via clipping)
PD_HIGH = 150.0  # MW (per-bus upper bound, enforced via clipping)
SEED = 77
TOTAL_DEMAND_FACTOR_LOW = 0.6  # 60% of baseline total demand
TOTAL_DEMAND_FACTOR_HIGH = 1.3  # 130% of baseline total demand

REGENERATE_SAMPLES = True  # force regeneration of PD chunks

samples_dir = Path.cwd() / "samples"
samples_dir.mkdir(parents=True, exist_ok=True)


# ---------------------------------------------------------------------------
# Helpers
def _chunk_name(end_count: int) -> str:
    return f"pd_chunk_{end_count:06d}.npz"


def _label_name(end_count: int) -> str:
    return f"labels_chunk_{end_count:06d}.npz"


def _get_baseline_pd(nb: int) -> np.ndarray:
    """Return baseline PD_MW from case118 (length nb)."""
    try:
        from pypower.api import case118

        ppc_local = case118()
        pd = ppc_local["bus"][:, 2].astype(np.float64)
        if pd.shape[0] != nb:
            raise ValueError(f"Baseline PD length {pd.shape[0]} != nb {nb}")
        return pd
    except Exception as e:
        raise RuntimeError(f"Failed to load baseline PD: {e}")


def generate_pd_chunks_lhs(nb: int):
    """
    Generate PD_MW samples using Latin Hypercube Sampling (LHS).

    Strategy:
    1. Use LHS to sample in [0,1]^(nb+1) space
    2. First dimension maps to total demand factor [TOTAL_DEMAND_FACTOR_LOW, TOTAL_DEMAND_FACTOR_HIGH]
    3. Remaining nb dimensions represent bus load fractions (normalized to sum=1)
    4. Each bus load = total demand × bus fraction
    5. Clip each bus load to [PD_LOW, PD_HIGH] MW

    Args:
        nb: number of buses
    """
    PD_base = _get_baseline_pd(nb)
    base_total = float(PD_base.sum())

    print(f"Baseline total demand: {base_total:.2f} MW")
    print(
        f"Target total demand range: [{base_total * TOTAL_DEMAND_FACTOR_LOW:.2f}, {base_total * TOTAL_DEMAND_FACTOR_HIGH:.2f}] MW"
    )
    print(f"Per-bus demand range: [{PD_LOW:.2f}, {PD_HIGH:.2f}] MW")
    print(f"Generating {TOTAL_SAMPLES} PD_MW samples using LHS with seed={SEED}...\n")

    # Use LHS to sample: nb+1 dimensions (1 for total factor, nb for bus fractions)
    sampler = qmc.LatinHypercube(d=nb + 1, seed=SEED)
    lhs_samples = sampler.random(n=TOTAL_SAMPLES)  # shape: (TOTAL_SAMPLES, nb+1)

    # First column maps to total demand factor
    total_factors = (
        lhs_samples[:, 0] * (TOTAL_DEMAND_FACTOR_HIGH - TOTAL_DEMAND_FACTOR_LOW)
        + TOTAL_DEMAND_FACTOR_LOW
    )

    # Remaining nb columns are bus load fractions
    bus_fractions = lhs_samples[:, 1:]  # shape: (TOTAL_SAMPLES, nb)

    # Normalize each row's fractions to sum to 1
    bus_fractions = bus_fractions / bus_fractions.sum(axis=1, keepdims=True)

    # Calculate target total for each sample
    target_totals = total_factors * base_total  # shape: (TOTAL_SAMPLES,)

    # Calculate per-bus loads: total × fraction
    PD_all = bus_fractions * target_totals[:, np.newaxis]  # shape: (TOTAL_SAMPLES, nb)

    # Clip per-bus loads to [PD_LOW, PD_HIGH]
    PD_all = np.clip(PD_all, PD_LOW, PD_HIGH).astype(np.float64)

    # Save in chunks
    for i in range(TOTAL_SAMPLES // CHUNK_SIZE):
        start = i * CHUNK_SIZE
        end = (i + 1) * CHUNK_SIZE
        chunk = PD_all[start:end, :]
        out_path = samples_dir / _chunk_name(end)
        np.savez_compressed(out_path, PD_MW=chunk)
        print(f"  Saved {CHUNK_SIZE} samples -> {out_path.name}")
    print("Done generating LHS PD_MW chunks.\n")


def build_model_once(frozen, PD_MW_init: np.ndarray):
    ng = int(frozen["ng"])
    nb = int(frozen["nb"])
    nbr = int(frozen["nbr"])
    m = build_concrete_dcopf(
        ng,
        nb,
        nbr,
        frozen["c2"],
        frozen["c1"],
        frozen["c0"],
        frozen["PGmin"],
        frozen["PGmax"],
        frozen["PLmin"],
        frozen["PLmax"],
        frozen["MG"],
        frozen["MD"],
        PD_MW_init,
    )
    return m


def set_pd_on_model(m, PD_MW_vec: np.ndarray):
    for d in m.D:
        m.PD[d] = float(PD_MW_vec[d])


def solve_and_label_chunk(frozen, chunk_path: Path, solver):
    ng = int(frozen["ng"])
    nb = int(frozen["nb"])

    data = np.load(chunk_path)
    PD_MW_batch = data["PD_MW"].astype(np.float64)
    n_batch = PD_MW_batch.shape[0]

    # Build model once with zeros; reuse and update PD each solve
    m = build_model_once(frozen, np.zeros(nb, dtype=np.float64))

    obj_arr = np.full(n_batch, np.nan, dtype=np.float64)
    PG_arr = np.full((n_batch, ng), np.nan, dtype=np.float64)
    term_arr = np.empty(n_batch, dtype=object)
    status_arr = np.empty(n_batch, dtype=object)
    feasible_arr = np.zeros(n_batch, dtype=bool)

    t0 = perf_counter()
    for k in range(n_batch):
        set_pd_on_model(m, PD_MW_batch[k])
        results = solver.solve(m, tee=False)
        term_cond = results.solver.termination_condition
        status = results.solver.status
        term_arr[k] = str(term_cond)
        status_arr[k] = str(status)
        # Mark feasible when solver reached (locally) optimal solution
        feasible_arr[k] = term_cond in (
            pyo.TerminationCondition.optimal,
            pyo.TerminationCondition.locallyOptimal,
        )
        if feasible_arr[k]:
            obj_arr[k] = float(pyo.value(m.OBJ))
            PG_arr[k, :] = np.array([pyo.value(m.PG[g]) for g in m.G], dtype=np.float64)
        if (k + 1) % 50 == 0 or (k + 1) == n_batch:
            print(f"    Labeled {k+1}/{n_batch}")
    dt = perf_counter() - t0
    print(f"  Chunk labeling done in {dt:.2f}s: {chunk_path.name}")

    end_count = int(chunk_path.stem.split("_")[-1])
    label_path = samples_dir / _label_name(end_count)
    np.savez_compressed(
        label_path,
        obj=obj_arr,
        PG=PG_arr,
        PD_MW=PD_MW_batch,
        term=term_arr,
        status=status_arr,
        feasible=feasible_arr,
    )
    print(f"  Saved labels -> {label_path.name}\n")


# ---------------------------------------------------------------------------
# Main routine
def main():
    frozen_path = Path.cwd() / "dcopf_frozen_params.npz"
    if not frozen_path.exists():
        raise FileNotFoundError(f"Frozen params file not found: {frozen_path}")
    frozen = np.load(frozen_path)
    ng = int(frozen["ng"])
    nb = int(frozen["nb"])
    nbr = int(frozen["nbr"])
    print("Loaded frozen parameters:")
    print(f"  ng={ng}, nb={nb}, nbr={nbr}, baseMVA={float(frozen['baseMVA'])}")
    print(f"  MG shape={frozen['MG'].shape}, MD shape={frozen['MD'].shape}\n")

    # Generate PD chunks using LHS
    if REGENERATE_SAMPLES:
        generate_pd_chunks_lhs(nb)
    else:
        need_generate = False
        for i in range(TOTAL_SAMPLES // CHUNK_SIZE):
            end = (i + 1) * CHUNK_SIZE
            chk = samples_dir / _chunk_name(end)
            if not chk.exists():
                need_generate = True
                break
        if need_generate:
            generate_pd_chunks_lhs(nb)
        else:
            print("All PD_MW chunk files already exist; skipping generation.\n")

    # Label all chunks
    solver = pyo.SolverFactory("gurobi")
    for i in range(TOTAL_SAMPLES // CHUNK_SIZE):
        end = (i + 1) * CHUNK_SIZE
        chunk_path = samples_dir / _chunk_name(end)
        if not chunk_path.exists():
            print(f"Missing chunk file: {chunk_path}")
            continue
        print(f"Labeling chunk: {chunk_path.name}")
        solve_and_label_chunk(frozen, chunk_path, solver)
    print("All chunks processed.")


# Run the main routine
main()


Loaded frozen parameters:
  ng=54, nb=118, nbr=186, baseMVA=100.0
  MG shape=(186, 54), MD shape=(186, 118)

Baseline total demand: 4242.00 MW
Target total demand range: [2545.20, 5514.60] MW
Per-bus demand range: [0.00, 150.00] MW
Generating 10000 PD_MW samples using LHS with seed=77...

  Saved 250 samples -> pd_chunk_000250.npz
  Saved 250 samples -> pd_chunk_000500.npz
  Saved 250 samples -> pd_chunk_000750.npz
  Saved 250 samples -> pd_chunk_001000.npz
  Saved 250 samples -> pd_chunk_001250.npz
  Saved 250 samples -> pd_chunk_001500.npz
  Saved 250 samples -> pd_chunk_001750.npz
  Saved 250 samples -> pd_chunk_002000.npz
  Saved 250 samples -> pd_chunk_002250.npz
  Saved 250 samples -> pd_chunk_002500.npz
  Saved 250 samples -> pd_chunk_002750.npz
  Saved 250 samples -> pd_chunk_002000.npz
  Saved 250 samples -> pd_chunk_002250.npz
  Saved 250 samples -> pd_chunk_002500.npz
  Saved 250 samples -> pd_chunk_002750.npz
  Saved 250 samples -> pd_chunk_003000.npz
  Saved 250 samples ->

### 样本状态检查

In [12]:
# Status summary for labeled chunks in ./samples
from pathlib import Path
import numpy as np
from collections import Counter

samples_dir = Path.cwd() / "samples"
label_files = sorted(samples_dir.glob("labels_chunk_*.npz"))

if not label_files:
    print(f"No label files found in {samples_dir}. Run cell 17 first to generate and label samples.")
else:
    grand_total = 0
    grand_feasible = 0
    term_counter = Counter()
    status_counter = Counter()
    obj_nan_infeas = 0
    obj_nan_feas = 0

    print(f"Found {len(label_files)} label files:\n  - " + "\n  - ".join(p.name for p in label_files))
    print("\nPer-chunk summary:")
    for p in label_files:
        data = np.load(p, allow_pickle=True)  # term/status are saved as object dtype
        feas = data["feasible"].astype(bool)
        terms = data["term"].astype(str)
        stats = data["status"].astype(str)
        obj = data["obj"].astype(float)

        n = feas.size
        n_feas = int(feas.sum())
        n_infeas = int(n - n_feas)
        grand_total += n
        grand_feasible += n_feas

        # Counters
        term_counter.update(terms.tolist())
        status_counter.update(stats.tolist())

        # Sanity check on obj NaNs
        obj_nan_infeas += int(np.isnan(obj[~feas]).sum())
        obj_nan_feas   += int(np.isnan(obj[feas]).sum())

        rate = (n_feas / n * 100.0) if n > 0 else 0.0
        print(f"  {p.name}: N={n}, feasible={n_feas}, infeasible={n_infeas}, feasible_rate={rate:5.1f}%")

    print("\nOverall summary:")
    overall_rate = (grand_feasible / grand_total * 100.0) if grand_total > 0 else 0.0
    print(f"  Total samples: {grand_total}")
    print(f"  Feasible:      {grand_feasible}")
    print(f"  Infeasible:    {grand_total - grand_feasible}")
    print(f"  Feasible rate: {overall_rate:5.1f}%")
    print("\nTermination conditions (top 10):")
    for term, cnt in term_counter.most_common(10):
        print(f"  {term:<30} {cnt}")
    print("\nSolver status (top 10):")
    for st, cnt in status_counter.most_common(10):
        print(f"  {st:<30} {cnt}")
    print("\nSanity checks:")
    print(f"  obj is NaN for infeasible rows: {obj_nan_infeas}")
    print(f"  obj is NaN for feasible rows:   {obj_nan_feas}")

Found 40 label files:
  - labels_chunk_000250.npz
  - labels_chunk_000500.npz
  - labels_chunk_000750.npz
  - labels_chunk_001000.npz
  - labels_chunk_001250.npz
  - labels_chunk_001500.npz
  - labels_chunk_001750.npz
  - labels_chunk_002000.npz
  - labels_chunk_002250.npz
  - labels_chunk_002500.npz
  - labels_chunk_002750.npz
  - labels_chunk_003000.npz
  - labels_chunk_003250.npz
  - labels_chunk_003500.npz
  - labels_chunk_003750.npz
  - labels_chunk_004000.npz
  - labels_chunk_004250.npz
  - labels_chunk_004500.npz
  - labels_chunk_004750.npz
  - labels_chunk_005000.npz
  - labels_chunk_005250.npz
  - labels_chunk_005500.npz
  - labels_chunk_005750.npz
  - labels_chunk_006000.npz
  - labels_chunk_006250.npz
  - labels_chunk_006500.npz
  - labels_chunk_006750.npz
  - labels_chunk_007000.npz
  - labels_chunk_007250.npz
  - labels_chunk_007500.npz
  - labels_chunk_007750.npz
  - labels_chunk_008000.npz
  - labels_chunk_008250.npz
  - labels_chunk_008500.npz
  - labels_chunk_008750.np

## 神经网络模型训练

### 基于 Pytorch PD (118) → PG (54) 的模型
目标值会使用 PGmin/PGmax 进行归一化；损失函数为归一化后的 PG 的均方误差（MSE）；优化器使用 Adam（学习率为 1e-4）。结果将保存到 `./results/` 目录下。

- 输入归一化公式：

$$
\text{Normalized input} = \frac{\text{Input} - \mu}{\sigma}
$$

其中：

- $\text{Input}$ 是给定总线的原始值（来自 `X_train` 或 `X_val`）。
- $\mu$ 是该总线的均值（从 `X_train` 计算得出）。
- $\sigma$ 是该总线的标准差（从 `X_train` 计算得出）。

- 目标归一化公式：

$$
\text{Normalized target} = \frac{\text{Target} - \text{PGmin}}{\text{PGmax} - \text{PGmin}}
$$

其中：

- $\text{Target}$ 是 `PG` 的原始值（给定发电机的发电功率）。
- $\text{PGmin}$ 是预定义的最小目标值。
- $\text{PGmax}$ 是预定义的最大目标值。

**目的**：此转换将目标值缩放到 [0, 1] 范围内，这有助于像神经网络这样的模型，因为它减少了大值对模型学习过程的影响。


In [13]:
# [PD->PG] Data prep and loaders
import os, json, math, random
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Paths
root = Path.cwd()
samples_dir = root / 'samples'
results_dir = root / 'results'
results_dir.mkdir(parents=True, exist_ok=True)

# Load frozen params (for dimensions and PG bounds)
frozen = np.load(root / 'dcopf_frozen_params.npz')
ng = int(frozen['ng']); nb = int(frozen['nb'])
PGmin = frozen['PGmin'].astype(np.float64)
PGmax = frozen['PGmax'].astype(np.float64)
assert PGmin.shape[0] == ng and PGmax.shape[0] == ng, 'PG bounds mismatch'
print(f"ng={ng}, nb={nb}")

# Aggregate feasible samples
label_files = sorted(samples_dir.glob('labels_chunk_*.npz'))
if not label_files:
    raise FileNotFoundError(f'No label files found in {samples_dir}. Generate them first.')

X_list = []  # PD (N, nb)
Y_list = []  # PG (N, ng)
keep = 0
for p in label_files:
    data = np.load(p, allow_pickle=True)
    feas = data['feasible'].astype(bool)
    if not np.any(feas):
        continue
    PD = data['PD_MW'].astype(np.float64)
    PG = data['PG'].astype(np.float64)
    # select feasible rows only
    sel = feas
    X_list.append(PD[sel])
    Y_list.append(PG[sel])
    keep += int(sel.sum())

if keep == 0:
    raise RuntimeError('No feasible samples found across label files.')

X = np.concatenate(X_list, axis=0)  # (N, nb)
Y = np.concatenate(Y_list, axis=0)  # (N, ng)
N = X.shape[0]
print(f"Collected feasible samples: N={N}, X={X.shape}, Y={Y.shape}")

# Train/val split
SEED = 77
rng = np.random.default_rng(SEED)
indices = np.arange(N)
rng.shuffle(indices)
val_frac = 0.1
n_val = int(round(val_frac * N))
val_idx = indices[:n_val]
train_idx = indices[n_val:]

X_train = X[train_idx]; X_val = X[val_idx]
Y_train = Y[train_idx]; Y_val = Y[val_idx]

# Normalize inputs PD by train mean/std (per-bus)
eps = 1e-8
PD_mean = X_train.mean(axis=0)
PD_std  = X_train.std(axis=0)
PD_std  = np.where(PD_std < 1e-6, 1.0, PD_std)  # guard small std
def norm_pd(arr: np.ndarray) -> np.ndarray:
    return (arr - PD_mean) / PD_std

# Normalize targets PG using PGmin/PGmax (per-generator min-max)
PG_den = (PGmax - PGmin)
PG_den = np.where(PG_den < 1e-6, 1.0, PG_den)
def norm_pg(arr: np.ndarray) -> np.ndarray:
    return (arr - PGmin) / PG_den
def denorm_pg(arr: np.ndarray) -> np.ndarray:
    return arr * PG_den + PGmin

X_train_n = norm_pd(X_train).astype(np.float32)
X_val_n   = norm_pd(X_val).astype(np.float32)
Y_train_n = norm_pg(Y_train).astype(np.float32)
Y_val_n   = norm_pg(Y_val).astype(np.float32)

# Torch datasets/loaders (robust to environments without numpy bridge in torch)
def to_tensor_cpu(x: np.ndarray) -> torch.Tensor:
    try:
        return torch.from_numpy(x)
    except Exception:
        # Fallback if torch's NumPy bridge is unavailable
        return torch.tensor(x.tolist(), dtype=torch.float32)

BATCH_SIZE = 20
train_ds = TensorDataset(to_tensor_cpu(X_train_n), to_tensor_cpu(Y_train_n))
val_ds   = TensorDataset(to_tensor_cpu(X_val_n),   to_tensor_cpu(Y_val_n))
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=(device.type=='cuda'), num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, pin_memory=(device.type=='cuda'), num_workers=0)

# Save normalization stats for inference
np.savez_compressed(results_dir / 'norm_stats.npz',
                     PD_mean=PD_mean.astype(np.float64),
                     PD_std=PD_std.astype(np.float64),
                     PGmin=PGmin.astype(np.float64),
                     PGmax=PGmax.astype(np.float64),
                     nb=nb, ng=ng)
print(f"Saved normalization stats to {results_dir / 'norm_stats.npz'}")

print(f"Train: {len(train_ds)} samples, Val: {len(val_ds)} samples, batch={BATCH_SIZE}")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\DevTools\anaconda3\envs\opf311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\DevTools\anaconda3\envs\opf311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\DevTools\anaconda3\envs\opf311\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "e:\DevTools

Using device: cuda
CUDA device: NVIDIA GeForce RTX 3070 Ti Laptop GPU
ng=54, nb=118
Collected feasible samples: N=10000, X=(10000, 118), Y=(10000, 54)
Saved normalization stats to e:\DOCUMENT\Learn_Py\opf\Week3\results\norm_stats.npz
Train: 9000 samples, Val: 1000 samples, batch=20
Saved normalization stats to e:\DOCUMENT\Learn_Py\opf\Week3\results\norm_stats.npz
Train: 9000 samples, Val: 1000 samples, batch=20


In [14]:
# [PD->PG] MLP model + training
import torch.nn as nn
import torch.optim as optim
from typing import Dict, List

class MLP_PD2PG(nn.Module):
    def __init__(self, in_dim: int, hidden: List[int], out_dim: int):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, out_dim))
        # Note: no output activation; targets are in [0,1] but MSE works with linear output.
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

def train_model(model: nn.Module, train_loader, val_loader, *, epochs: int = 100, lr: float = 1e-4, device: torch.device = device) -> Dict[str, List[float]]:
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()  # MSE on normalized PG
    history = {"train_loss": [], "val_loss": []}
    best_val = float('inf')
    best_path = results_dir / 'model_mlp_pg_from_pd.pt'

    for epoch in range(1, epochs+1):
        model.train()
        running = 0.0
        n_train = 0
        for xb, yb in train_loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            running += loss.item() * xb.size(0)
            n_train += xb.size(0)
        train_loss = running / max(1, n_train)

        # Validation
        model.eval()
        running_v = 0.0
        n_val = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device, non_blocking=True)
                yb = yb.to(device, non_blocking=True)
                pred = model(xb)
                loss = criterion(pred, yb)
                running_v += loss.item() * xb.size(0)
                n_val += xb.size(0)
        val_loss = running_v / max(1, n_val)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        if val_loss < best_val:
            best_val = val_loss
            torch.save({
                'model_state': model.state_dict(),
                'in_dim': nb, 'hidden': [50,50,50], 'out_dim': ng,
                'normalize': 'PG min-max; PD z-score',
                'device': str(device)
            }, best_path)
        if epoch % 5 == 0 or epoch == 1 or epoch == epochs:
            print(f"Epoch {epoch:3d}/{epochs}  train={train_loss:.6f}  val={val_loss:.6f}")

    # Save history
    with open(results_dir / 'training_log.json', 'w', encoding='utf-8') as f:
        json.dump(history, f)
    print(f"Saved model to {best_path} and history to training_log.json")
    return history

# Configure and (optionally) train
RUN_TRAINING = True  # Set True to train for full epochs
EPOCHS = 100
LR = 1e-4

model = MLP_PD2PG(nb, [50,50,50], ng)
if RUN_TRAINING:
    hist = train_model(model, train_loader, val_loader, epochs=EPOCHS, lr=LR, device=device)
    # Save a few sample predictions on validation set (denormalized to MW)
    model.eval()
    xb, yb = next(iter(val_loader))
    xb = xb.to(device)
    with torch.no_grad():
        y_pred_t = model(xb).cpu()
    # Convert to NumPy robustly
    try:
        y_pred_n = y_pred_t.numpy()
        y_true_n = yb.numpy()
    except Exception:
        # Fallback if NumPy bridge unavailable
        y_pred_n = np.array(y_pred_t.tolist(), dtype=np.float32)
        y_true_n = np.array(yb.tolist(), dtype=np.float32)
    # Denormalize to MW
    y_pred = (y_pred_n * PG_den + PGmin).astype(np.float64)
    y_true = (y_true_n * PG_den + PGmin).astype(np.float64)
    np.savez_compressed(results_dir / 'sample_predictions.npz',
                         y_true_MW=y_true, y_pred_MW=y_pred)
    print(f"Saved sample_predictions.npz to {results_dir}")

Epoch   1/100  train=0.047086  val=0.009492
Epoch   5/100  train=0.002335  val=0.002302
Epoch   5/100  train=0.002335  val=0.002302
Epoch  10/100  train=0.000754  val=0.000775
Epoch  10/100  train=0.000754  val=0.000775
Epoch  15/100  train=0.000275  val=0.000306
Epoch  15/100  train=0.000275  val=0.000306
Epoch  20/100  train=0.000132  val=0.000155
Epoch  20/100  train=0.000132  val=0.000155
Epoch  25/100  train=0.000071  val=0.000089
Epoch  25/100  train=0.000071  val=0.000089
Epoch  30/100  train=0.000047  val=0.000063
Epoch  30/100  train=0.000047  val=0.000063
Epoch  35/100  train=0.000034  val=0.000051
Epoch  35/100  train=0.000034  val=0.000051
Epoch  40/100  train=0.000027  val=0.000043
Epoch  40/100  train=0.000027  val=0.000043
Epoch  45/100  train=0.000021  val=0.000037
Epoch  45/100  train=0.000021  val=0.000037
Epoch  50/100  train=0.000018  val=0.000033
Epoch  50/100  train=0.000018  val=0.000033
Epoch  55/100  train=0.000015  val=0.000031
Epoch  55/100  train=0.000015  v

In [15]:
# [PD->PG] Quick sanity check (no long training)
# - Verifies shapes and a forward/backward pass on one mini-batch.
import torch.nn as nn

def sanity_step():
    tmp_model = MLP_PD2PG(nb, [50,50,50], ng).to(device)
    xb, yb = next(iter(train_loader))
    xb = xb.to(device)
    yb = yb.to(device)
    pred = tmp_model(xb)
    assert pred.shape == yb.shape, f"Pred shape {pred.shape} != target {yb.shape}"
    loss = nn.MSELoss()(pred, yb)
    loss.backward()
    print(f"Sanity OK: batch={xb.shape[0]}, loss={loss.item():.6f}, device={device}")

sanity_step()

Sanity OK: batch=20, loss=0.150221, device=cuda


### 基于 `case118.py` 的模型输出测试
Run the trained MLP on the original case118 bus demands and inspect the predicted generator outputs (MW), including power balance and bound checks.

In [16]:
# Baseline inference code
import numpy as np
import torch
from pathlib import Path
from pypower.api import case118
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root = Path.cwd()
results_dir = root / 'results'

# Load normalization stats
stats = np.load(results_dir / 'norm_stats.npz')
PD_mean = stats['PD_mean'].astype(np.float64)
PD_std  = stats['PD_std'].astype(np.float64)
PGmin   = stats['PGmin'].astype(np.float64)
PGmax   = stats['PGmax'].astype(np.float64)
nb = int(stats['nb']); ng = int(stats['ng'])
PG_den = np.where((PGmax - PGmin) < 1e-6, 1.0, (PGmax - PGmin))

# Recreate model and load weights
ckpt = torch.load(results_dir / 'model_mlp_pg_from_pd.pt', map_location=device)
in_dim = int(ckpt.get('in_dim', nb))
hidden = ckpt.get('hidden', [50,50,50])
out_dim = int(ckpt.get('out_dim', ng))

import torch.nn as nn
class MLP_PD2PG(nn.Module):
    def __init__(self, in_dim, hidden, out_dim):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers += [nn.Linear(last, h), nn.ReLU()]
            last = h
        layers += [nn.Linear(last, out_dim)]
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

model = MLP_PD2PG(in_dim, hidden, out_dim).to(device)
model.load_state_dict(ckpt['model_state'])
model.eval()

# Load baseline Pd from case118 (MW)
ppc = case118()
PD_baseline = ppc['bus'][:, 2].astype(np.float64)
assert PD_baseline.shape[0] == nb, f'Pd length {PD_baseline.shape[0]} != nb {nb}'

# Normalize input
PD_norm = (PD_baseline - PD_mean) / np.where(PD_std < 1e-6, 1.0, PD_std)
x = torch.tensor(PD_norm, dtype=torch.float32, device=device).unsqueeze(0)  # (1, nb)

# Inference
with torch.no_grad():
    y_pred_n_t = model(x).squeeze(0).detach().cpu()
try:
    y_pred_n = y_pred_n_t.numpy()
except Exception:
    y_pred_n = np.array(y_pred_n_t.tolist(), dtype=np.float32)

# Denormalize to MW and clamp to bounds
PG_pred = (y_pred_n * PG_den + PGmin).astype(np.float64)
PG_pred = np.minimum(np.maximum(PG_pred, PGmin), PGmax)

# Summaries
total_PD = float(PD_baseline.sum())
total_PG = float(PG_pred.sum())
mismatch = total_PG - total_PD

print('=== Baseline PD -> Predicted PG (MW) ===')
print(f'Buses (nb) = {nb}, Generators (ng) = {ng}')
print(f'Total demand (MW):    {total_PD:,.2f}')
print(f'Total generation (MW): {total_PG:,.2f}')
print(f'Balance mismatch (MW): {mismatch:+.4f}')
print('\nFirst 10 generator outputs (MW):')
for i in range(min(10, ng)):
    print(f'  G{i:02d}: {PG_pred[i]:10.3f}   [min={PGmin[i]:.1f}, max={PGmax[i]:.1f}]')

# Optional: basic violation check (should be clamped)
viol_min = int((PG_pred < PGmin - 1e-6).sum())
viol_max = int((PG_pred > PGmax + 1e-6).sum())
print(f'\nBounds violations after clamp: below-min={viol_min}, above-max={viol_max}')

=== Baseline PD -> Predicted PG (MW) ===
Buses (nb) = 118, Generators (ng) = 54
Total demand (MW):    4,242.00
Total generation (MW): 4,289.05
Balance mismatch (MW): +47.0546

First 10 generator outputs (MW):
  G00:      7.198   [min=0.0, max=100.0]
  G01:      7.045   [min=0.0, max=100.0]
  G02:      7.353   [min=0.0, max=100.0]
  G03:      6.982   [min=0.0, max=100.0]
  G04:    245.566   [min=0.0, max=550.0]
  G05:     87.622   [min=0.0, max=185.0]
  G06:      7.034   [min=0.0, max=100.0]
  G07:      7.631   [min=0.0, max=100.0]
  G08:      7.623   [min=0.0, max=100.0]
  G09:      3.880   [min=0.0, max=100.0]

Bounds violations after clamp: below-min=0, above-max=0
